# Segmenting and Clustering Neighborhoods in Toronto


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


# Get Data 

In [2]:
# Link to table on wikipedia
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# Get data
df = pd.read_html(link,header=0)[0]
# Show dataFrame
df.head()

Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

# Cleaning data

In [3]:
# Remove rows with value 'Not assigned'for Borough
df=df[df['Borough']!='Not assigned']
df.reset_index(drop=True, inplace=True)
df.head()

Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [4]:
# Check for More than one neighborhood can exist in one postal code area
(df.groupby('Postal Code')
   .agg({'Borough' : 'first', 'Neighborhood' : ','.join})
   .reset_index()
   .reindex(columns=df.columns))
df.head()

Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

## Shape of DataFrame -  <span style="color:red"> First Task ends here text</span>

In [5]:
# Dataframe shape
df.shape

(103, 3)

# Geospatial data

In [6]:
linkGeo = 'https://cocl.us/Geospatial_data'
# Get data
dfGeo = pd.read_csv(linkGeo)
# Show dataFrame
dfGeo.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

# Join DataFrames - <span style="color:red"> Second Task ends here text</span>

In [7]:
dfNew=df.join(dfGeo.set_index('Postal Code'),on='Postal Code')

In [8]:
#Final DataFrame
dfNew

Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
5           M9A         Etobicoke   
6           M1B       Scarborough   
7           M3B        North York   
8           M4B         East York   
9           M5B  Downtown Toronto   
10          M6B        North York   
11          M9B         Etobicoke   
12          M1C       Scarborough   
13          M3C        North York   
14          M4C         East York   
15          M5C  Downtown Toronto   
16          M6C              York   
17          M9C         Etobicoke   
18          M1E       Scarborough   
19          M4E      East Toronto   
20          M5E  Downtown Toronto   
21          M6E              York   
22          M1G       Scarborough   
23          M4G         East York   
24          M5G  Downtown Toronto   
25          M6G  Downtown Toronto   
26          M1H       Scarborough   
27          M2H        North York   
28          M3H        North York   
29          M4H         East York   
30          M5H  Downtown Toronto   
31          M6H      West Toronto   
32          M1J       Scarborough   
33          M2J        North York   
34          M3J        North York   
35          M4J         East York   
36          M5J  Downtown Toronto   
37          M6J      West Toronto   
38          M1K       Scarborough   
39          M2K        North York   
40          M3K        North York   
41          M4K      East Toronto   
42          M5K  Downtown Toronto   
43          M6K      West Toronto   
44          M1L       Scarborough   
45          M2L        North York   
46          M3L        North York   
47          M4L      East Toronto   
48          M5L  Downtown Toronto   
49          M6L        North York   
50          M9L        North York   
51          M1M       Scarborough   
52          M2M        North York   
53          M3M        North York   
54          M4M      East Toronto   
55          M5M        North York   
56          M6M              York   
57          M9M        North York   
58          M1N       Scarborough   
59          M2N        North York   
60          M3N        North York   
61          M4N   Central Toronto   
62          M5N   Central Toronto   
63          M6N              York   
64          M9N              York   
65          M1P       Scarborough   
66          M2P        North York   
67          M4P   Central Toronto   
68          M5P   Central Toronto   
69          M6P      West Toronto   
70          M9P         Etobicoke   
71          M1R       Scarborough   
72          M2R        North York   
73          M4R   Central Toronto   
74          M5R   Central Toronto   
75          M6R      West Toronto   
76          M7R       Mississauga   
77          M9R         Etobicoke   
78          M1S       Scarborough   
79          M4S   Central Toronto   
80          M5S  Downtown Toronto   
81          M6S      West Toronto   
82          M1T       Scarborough   
83          M4T   Central Toronto   
84          M5T  Downtown Toronto   
85          M1V       Scarborough   
86          M4V   Central Toronto   
87          M5V  Downtown Toronto   
88          M8V         Etobicoke   
89          M9V         Etobicoke   
90          M1W       Scarborough   
91          M4W  Downtown Toronto   
92          M5W  Downtown Toronto   
93          M8W         Etobicoke   
94          M9W         Etobicoke   
95          M1X       Scarborough   
96          M4X  Downtown Toronto   
97          M5X  Downtown Toronto   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656

# Create map of Toronto and Neighborhoods

In [9]:
address = 'Amsterdam, The Netherlands'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 52.3727598, 4.8936041.


In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfNew['Latitude'], dfNew['Longitude'], dfNew['Borough'], dfNew['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  

map_newyork

# Borough with ''Toronto'' in name

In [11]:
toronto_data = dfNew[dfNew['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

Postal Code           Borough                                 Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.676357 -79.293031

##### Create map of Toronto using latitude and longitude values

In [12]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  

map_newyork

#### Foursquare credentials

In [13]:
CLIENT_ID = 'TPZXZNA4ANEH4PIYSU41EYO1AJE3EF353UMULU5BYIKRT5TB' # your Foursquare ID
CLIENT_SECRET = 'UXLZAOUVWPJOPTSMJB1JNIKJ1G4AQDYMRSRAFO0HJ13RLIXY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TPZXZNA4ANEH4PIYSU41EYO1AJE3EF353UMULU5BYIKRT5TB
CLIENT_SECRET:UXLZAOUVWPJOPTSMJB1JNIKJ1G4AQDYMRSRAFO0HJ13RLIXY


##### Let's explore the neighborhood Christie

In [14]:
toronto_data.loc[7, 'Neighborhood']

'Christie'

##### Find coordinates for Chistie

In [15]:
neighborhood_latitude = toronto_data.loc[7, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[7, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[7, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Christie are 43.669542, -79.4225637.


### Lets take 100 venues in Christie within a radius of 1000 meters

In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

##### Function that extracts the category of the venue

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

name          categories        lat        lng
0           Fiesta Farms       Grocery Store  43.668471 -79.420485
1            Contra Cafe                Café  43.669107 -79.426105
2         Vinny’s Panini  Italian Restaurant  43.670679 -79.426148
3              Starbucks         Coffee Shop  43.671530 -79.421400
4   Scout and Cash Caffe                Café  43.667360 -79.419938
5        Universal Grill               Diner  43.670550 -79.426541
6            Faema Caffe                Café  43.671046 -79.419297
7             Actinolite          Restaurant  43.667858 -79.428054
8      Stubbe Chocolates         Candy Store  43.671566 -79.421289
9                Loblaws       Grocery Store  43.671657 -79.421364
10         Sobeys Dupont       Grocery Store  43.670977 -79.427573
11  Marlenes Just Babies          Baby Store  43.671824 -79.420499
12          Dupont Disco           Nightclub  43.670490 -79.426611
13     Marian Engel Park                Park  43.673754 -79.423988
14   Garrison Creek Park                Park  43.671690 -79.427805
15          Foto Grocery       Grocery Store  43.667979 -79.428217

In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


## Explore neighborhoods where the borough has Toronto in the name

##### Function to get data

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Get Data

In [22]:

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

##### Size of resulting dataFrame

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(1618, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2      Morning Glory Cafe       43.653947       -79.361149   
3  Cooper Koo Family YMCA       43.653249       -79.358008   
4     Body Blitz Spa East       43.654735       -79.359874   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2       Breakfast Spot  
3  Distribution Center  
4                  Spa

#### Number of venus by neighborhood

In [26]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            56   
Brockton, Parkdale Village, Exhibition Place                           23   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     18   
Central Bay Street                                                     65   
Christie                                                               16   
Church and Wellesley                                                   78   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             31   
Davisville North                                                        9   
Dufferin, Dovercourt Village                                           15   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          23   
India Bazaar, The Beaches West                                         21   
Kensington Market, Chinatown, Grange Park                              60   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               45   
Moore Park, Summerhill East                                             2   
North Toronto West, Lawrence Park                                      19   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            32   
Regent Park, Harbourfront                                              46   
Richmond, Adelaide, King                                               95   
Rosedale                                                                4   
Roselawn                                                                3   
Runnymede, Swansea                                                     38   
St. James Town                                                         78   
St. James Town, Cabbagetown                                            48   
Stn A PO Boxes                                                         94   
Studio District                                                        41   
Summerhill West, Rathnelly, South Hill, Forest ...                     17   
The Annex, North Midtown, Yorkville                                    22   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         35   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             56   
Brockton, Parkdale Village, Exhibition Place                            23   
Business reply mail Processing Centre, South Ce...                      16   
CN Tower, King and Spadina, Railway Lands, Harb...                      18   
Central Bay Street                                                      65   
Christie                                                                16   
Church and Wellesley                                                    78   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              31   
Davisville North                    

##### Unique categories 

In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


## Analyze each neighborhood

In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
first_col = toronto_onehot.pop("Neighborhood")
toronto_onehot.insert(0,"Neighborhood",first_col)
toronto_onehot.head()

Neighborhood  Afghan Restaurant  Airport  Airport Food Court  \
0  Regent Park, Harbourfront                  0        0                   0   
1  Regent Park, Harbourfront                  0        0                   0   
2  Regent Park, Harbourfront                  0        0                   0   
3  Regent Park, Harbourfront                  0        0                   0   
4  Regent Park, Harbourfront                  0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  Auto Workshop  BBQ Joint  \
0                    0                 0              0          0   
1                    0                 0              0          0   
2                    0                 0              0          0   
3                    0                 0              0          0   
4                    0                 0              0          0   

   Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  \
0           0           0       1     0    0                 0   
1           0           0       0     0    0                 0   
2           0           0       0     0    0                 0   
3           0           0       0     0    0                 0   
4           0           0       0     0    0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0       0              0          0         0   
1                   0       0              0          0         0   
2                   0       0              0          0         0   
3                   0       0              0          0         0   
4                   0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  Building  \
0                     0               0        0                0         0   
1                     0               0        0                0         0   
2                     0               1        0                0         0   
3                     0               0        0                0         0   
4                     0               0        0                0         0   

   Burger Joint  Burrito Place  Bus Line  Butcher  Café  \
0             0              0         0        0     0   
1             0              0         0        0     0   
2             0              0         0        0     0   
3             0              0         0        0     0   
4             0              0         0        0     0   

   Cajun / Creole Restaurant  Camera Store  Candy Store  Caribbean Restaurant  \
0                          0             0            0                     0   
1          

In [37]:
# Size of dataFrame
toronto_onehot.shape

(1618, 237)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Afghan Restaurant  \
0                                        Berczy Park                0.0   
1       Brockton, Parkdale Village, Exhibition Place                0.0   
2  Business reply mail Processing Centre, South C...                0.0   
3  CN Tower, King and Spadina, Railway Lands, Har...                0.0   
4                                 Central Bay Street                0.0   

    Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0  0.000000            0.000000      0.000000        0.000000   
1  0.000000            0.000000      0.000000        0.000000   
2  0.000000            0.000000      0.000000        0.000000   
3  0.055556            0.055556      0.055556        0.111111   
4  0.000000            0.000000      0.000000        0.000000   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0         0.000000          0.000000                  0.0           0.0   
1         0.000000          0.000000                  0.0           0.0   
2         0.000000          0.000000                  0.0           0.0   
3         0.166667          0.111111                  0.0           0.0   
4         0.000000          0.000000                  0.0           0.0   

   Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0       0.0     0.017857    0.000000                  0.0               0.0   
1       0.0     0.000000    0.000000                  0.0               0.0   
2       0.0     0.000000    0.000000                  0.0               0.0   
3       0.0     0.000000    0.000000                  0.0               0.0   
4       0.0     0.000000    0.015385                  0.0               0.0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop    Bakery  Bank       Bar  \
0         0.0000   0.017857         0.0    0.017857  0.035714   0.0  0.000000   
1         0.0000   0.000000         0.0    0.000000  0.086957   0.0  0.043478   
2         0.0625   0.000000         0.0    0.000000  0.000000   0.0  0.000000   
3         0.0000   0.000000         0.0    0.000000  0.000000   0.0  0.055556   
4         0.0000   0.000000         0.0    0.000000  0.000000   0.0  0.015385   

   Baseball Stadium  Basketball Stadium     Beach  Bed & Breakfast  Beer Bar  \
0               0.0            0.017857  0.017857              0.0  0.035714   
1               0.0            0.000000  0.000000              0.0  0.000000   
2               0.0            0.000000  0.000000              0.0  0.000000   
3               0.0            0.000000  0.000000              0.0  0.000000   
4               0.0            0.000000  0.000000              0.0  0.000000   

   Beer Store  Belgian Restaurant    Bistro  Boat or Ferry  Bookstore  \
0         0.0                 0.0  0.017857       0.000000        0.0   
1         0.0                 0.0  0.000000       0.000000        0.0   
2         0.0                 0.0  0.000000       0.000000        0.0   
3         0.0                 0.0  0.000000       0.055556        0.0   
4         0.0                 0.0  0.000000       0.000000        0.0   

   Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  \
0  0.000000                   0.0        0.017857   0.0000         0.000000   
1  0.000000                   0.0        0.086957   0.0000         0.000000   
2  0.000000                   0.0        0.000000   0.0625         0.000000   
3  0.055556                   0.0        0.000000   0.0000         0.000000   
4  0.000000                   0.0        0.000000   0.0000         0.030769   

   Building  Burger Joint  Burrito Place  Bus Line  Butcher      Café  \
0       0.0      0.000000       0.000000       0.0   0.0000  0.035714   
1       0.0      0.000000       0.043478       0.0   0.0000  0.130435   
2       0.0      0.000000       0.062500       0.0   0.0625  0.000000   
3       0.0      0.000000       0.000000       0.0   0.0000  0.000000   
4       0.0      0.030769

#### Let's confirm the new size

In [40]:
toronto_grouped.shape

(39, 237)

#### Let's print each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3            Beer Bar  0.04
4          Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
               venue  freq
0               Café  0.13
1     Breakfast Spot  0.09
2        Coffee Shop  0.09
3             Bakery  0.09
4  Convenience Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Gym / Fitness Center  0.06
1               Brewery  0.06
2            Skate Park  0.06
3            Restaurant  0.06
4           Pizza Place  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.17
1       Airport Lounge  0.11
2     Airport Terminal  0.11
3             Boutique  0.06
4  Rental Car Location  0.06


----Central Bay Street----


#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 20 venues for each neighborhood.

In [57]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...           Yoga Studio   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar           Cheese Shop              Beer Bar   
1           Coffee Shop        Breakfast Spot                Bakery   
2         Auto Workshop                  Park           Pizza Place   
3        Airport Lounge      Airport Terminal   Rental Car Location   
4                  Café    Italian Restaurant        Sandwich Place   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                Bakery    Seafood Restaurant            Restaurant   
1                   Gym               Stadium         Burrito Place   
2            Restaurant               Butcher         Burrito Place   
3       Harbor / Marina                 Plane         Boat or Ferry   
4   Japanese Restaurant      Department Store           Salad Place   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
0                  Café        Clothing Store              Irish Pub   
1            Restaurant          Climbing Gym              Pet Store   
2            Skate Park               Brewery             Comic Shop   
3              Boutique                   Bar       Sculpture Garden   
4          Burger Joint       Bubble Tea Shop      Indian Restaurant   

  11th Most Common Venue  12th Most Common Venue 13th Most Common Venue  \
0            Fish Market            Concert Hall                 Bistro   
1      Convenience Store               Nightclub     Italian Restaurant   
2         Farmers Market    Fast Food Restaurant     Light Rail Station   
3           Airport Gate      Airport Food Court                Airport   
4                  Hotel  Furniture / Home Store      French Restaurant   

  14th Most Common Venue       15th Most Common Venue 16th Most Common Venue  \
0                  Hotel                       Museum         Farmers Market   
1           Intersection       Furniture / Home Store          Grocery Store   
2   Gym / Fitness Center                Garden Center                 Garden   
3            Coffee Shop  Eastern European Restaurant       Doner Restaurant   
4   Gym / Fitness Center                   Restaurant       Ramen Restaurant   

    17th Most Common Venue 18th Most Common Venue  \
0  Comfort Food Restaurant         Breakfast Spot   
1                      Bar  Performing Arts Venue   
2   Furniture / Home Store  General Entertainment   
3               Donut Shop    Dumpling Restaurant   
4       Falafel Restaurant             Poke Place   

        19th Most Common Venue 20th Most Common Venue  
0  Eastern European Restaurant                   Park  
1                   Bagel Shop             Baby Store  
2               General Travel       Doner Restaurant  
3                  Yoga Studio      Electronics Store  
4           Chinese Restaurant                   Park

# Clustering

Run *k*-means to cluster the neighborhood into 10 clsters.

In [70]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=10).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([1, 1, 1, 8, 1, 0, 1, 1, 1, 9, 0, 1, 7, 1, 1, 0, 1, 1, 6, 1, 3, 1,
       1, 1, 1, 1, 2, 5, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [71]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Postal Code           Borough                                 Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.654260 -79.360636               1           Coffee Shop   
1  43.662301 -79.389494               1           Coffee Shop   
2  43.657162 -79.378937               1        Clothing Store   
3  43.651494 -79.375418               1           Coffee Shop   
4  43.676357 -79.293031               4                 Trail   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Park                Bakery                   Pub   
1      Sushi Restaurant        Discount Store        Sandwich Place   
2           Coffee Shop       Bubble Tea Shop   Japanese Restaurant   
3                  Café          Cocktail Bar             Gastropub   
4     Health Food Store                   Pub           Yoga Studio   

  5th Most Common Venue 6th Most Common Venue      7th Most Common Venue  \
0        Breakfast Spot                  Café                 Restaurant   
1                  Park           Music Venue         Mexican Restaurant   
2    Italian Restaurant                  Café  Middle Eastern Restaurant   
3   American Restaurant            Restaurant                      Hotel   
4                 Diner        Discount Store        Distribution Center   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
0               Theater        Farmers Market         Chocolate Shop   
1    Italian Restaurant            Hobby Shop    Fried Chicken Joint   
2        Cosmetics Shop                 Diner               Tea Room   
3    Italian Restaurant        Lingerie Store       Department Store   
4               Dog Run      Doner Restaurant             Donut Shop   

  11th Most Common Venue 12th Most Common Venue       13th Most Common Venue  \
0   Gym / Fitness Center             Shoe Store            Health Food Store   
1    Distribution Center            Yoga Studio                Smoothie Shop   
2       Ramen Restaurant                  Hotel                       Bakery   
3               Creperie         Cosmetics Shop          Moroccan Restaurant   
4           Dessert Shop    Dumpling Restaurant  Eastern European Restaurant   

  14th Most Common Venue 15th Most Common Venue 16th Most Common Venue  \
0          Historic Site      French Restaurant                  Hotel   
1               Creperie     College Auditorium                   Café   
2   Fast Food Restaurant              Bookstore                Theater   
3         Clothing Store                    Gym            Art Gallery   
4      Electronics Store   Ethiopian Restaurant            Event Space   

  17th Most Common Venue      18th Most Common Venue 19th Most Common Venue  \
0         Ice Cream Shop                 Event Space  Performing Arts Venue   
1          Burrito Place                    Beer Bar                    Bar   
2         Lingerie Store  Modern European Restaurant            Pizza Place   
3              Jazz Club             Thai Restaurant           Camera Store   
4     Dim Sum Restaurant               Deli / Bodega       Department Store   

  20th Most Common Venue  
0         Cosmetics Shop  
1                   Bank  
2                   Lake  
3             Poke Place  
4         Farmers Market

In [73]:
# Visualize clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Explore each cluster

In [75]:

cluster= 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
7   Downtown Toronto               0         Grocery Store   
9       West Toronto               0              Pharmacy   
22      West Toronto               0       Thai Restaurant   

   2nd Most Common Venue      3rd Most Common Venue 4th Most Common Venue  \
7                   Café                       Park           Coffee Shop   
9                 Bakery  Middle Eastern Restaurant                  Café   
22    Mexican Restaurant                       Café         Grocery Store   

        5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
7                   Nightclub           Candy Store            Baby Store   
9                         Bar           Supermarket                  Bank   
22  Cajun / Creole Restaurant   Fried Chicken Joint             Speakeasy   

   8th Most Common Venue   9th Most Common Venue 10th Most Common Venue  \
7     Italian Restaurant                   Diner             Restaurant   
9            Music Venue                 Brewery              Pet Store   
22    Italian Restaurant  Furniture / Home Store    Arts & Crafts Store   

   11th Most Common Venue 12th Most Common Venue 13th Most Common Venue  \
7                 Dog Run     Dim Sum Restaurant    Distribution Center   
9                    Park              Wine Shop          Grocery Store   
22                    Bar              Bookstore                   Park   

    14th Most Common Venue 15th Most Common Venue 16th Most Common Venue  \
7   Gluten-free Restaurant         Discount Store       Doner Restaurant   
9         Cuban Restaurant                  Diner      Electronics Store   
22                  Bakery           Antique Shop            Music Venue   

         17th Most Common Venue 18th Most Common Venue  \
7                    Donut Shop    Dumpling Restaurant   
9   Eastern European Restaurant    Dumpling Restaurant   
22                    Gastropub   Fast Food Restaurant   

         19th Most Common Venue 20th Most Common Venue  
7   Eastern European Restaurant              Gift Shop  
9                    Donut Shop       Doner Restaurant  
22                        Diner            Flea Market

In [76]:
cluster= 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0   Downtown Toronto               1           Coffee Shop   
1   Downtown Toronto               1           Coffee Shop   
2   Downtown Toronto               1        Clothing Store   
3   Downtown Toronto               1           Coffee Shop   
5   Downtown Toronto               1           Coffee Shop   
6   Downtown Toronto               1           Coffee Shop   
8   Downtown Toronto               1           Coffee Shop   
10  Downtown Toronto               1           Coffee Shop   
11      West Toronto               1                   Bar   
12      East Toronto               1      Greek Restaurant   
13  Downtown Toronto               1           Coffee Shop   
14      West Toronto               1                  Café   
15      East Toronto               1        Sandwich Place   
16  Downtown Toronto               1           Coffee Shop   
17      East Toronto               1                  Café   
23   Central Toronto               1        Clothing Store   
24   Central Toronto               1        Sandwich Place   
25      West Toronto               1        Breakfast Spot   
26   Central Toronto               1          Dessert Shop   
27  Downtown Toronto               1                  Café   
28      West Toronto               1           Coffee Shop   
30  Downtown Toronto               1                  Café   
31   Central Toronto               1           Coffee Shop   
34  Downtown Toronto               1           Coffee Shop   
35  Downtown Toronto               1                  Café   
36  Downtown Toronto               1           Coffee Shop   
37  Downtown Toronto               1           Coffee Shop   
38      East Toronto               1           Yoga Studio   

   2nd Most Common Venue  3rd Most Common Venue  \
0                   Park                 Bakery   
1       Sushi Restaurant         Discount Store   
2            Coffee Shop        Bubble Tea Shop   
3                   Café           Cocktail Bar   
5           Cocktail Bar            Cheese Shop   
6                   Café     Italian Restaurant   
8                   Café             Restaurant   
10              Aquarium                   Café   
11      Asian Restaurant                   Café   
12    Italian Restaurant            Coffee Shop   
13                 Hotel                   Café   
14           Coffee Shop         Breakfast Spot   
15  Fast Food Restaurant                    Gym   
16            Restaurant                   Café   
17           Coffee Shop                Brewery   
23           Coffee Shop            Yoga Studio   
24                  Café            Coffee Shop   
25             Gift Shop       Cuban Restaurant   
26        Sandwich Place       Sushi Restaurant   
27                   Bar     Italian Restaurant   
28      Sushi Restaurant                   Café   
30           Coffee Shop  Vietnamese Restaurant   
31    Light Rail Station                    Pub   
34                  Café     Italian Restaurant   
35           Coffee Shop            Pizza Place   
36                  Café                    Gym   
37      Sushi Restaurant    Japanese Restaurant   
38         Auto Workshop                   Park   

            4th Most Common Venue 5th Most Common Venue  \
0                             Pub        Breakfast Spot   
1                  Sandwich Place                  Park   
2             Japanese Restaurant    Italian Restaurant   
3                       Gastropub   American Restaurant   
5                        Beer Bar                Bakery   
6                  Sandwich Place   Japanese Restaurant   
8                             Gym         Deli / Bodega   
10                          Hotel            Restaurant   
11                    Men's Store            Restaurant   
12                      Bookstore            Restaurant   
13                     Restaurant    Italian Restaurant   
14                        

In [77]:
cluster= 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
33  Downtown Toronto               2                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
33            Playground                 Trail      Department Store   

   5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
33  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
33   Dumpling Restaurant            Donut Shop       Doner Restaurant   

   11th Most Common Venue 12th Most Common Venue 13th Most Common Venue  \
33                Dog Run    Distribution Center         Discount Store   

   14th Most Common Venue 15th Most Common Venue 16th Most Common Venue  \
33                  Diner     Dim Sum Restaurant           Dessert Shop   

   17th Most Common Venue 18th Most Common Venue 19th Most Common Venue  \
33          Deli / Bodega     Falafel Restaurant           Dance Studio   

   20th Most Common Venue  
33           Cupcake Shop

In [78]:
cluster= 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
29  Central Toronto               3                   Gym   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
29                  Park          Dessert Shop           Event Space   

   5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
29  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
29   Dumpling Restaurant            Donut Shop       Doner Restaurant   

   11th Most Common Venue 12th Most Common Venue 13th Most Common Venue  \
29                Dog Run    Distribution Center         Discount Store   

   14th Most Common Venue 15th Most Common Venue 16th Most Common Venue  \
29                  Diner     Dim Sum Restaurant       Department Store   

   17th Most Common Venue 18th Most Common Venue 19th Most Common Venue  \
29         Farmers Market          Deli / Bodega           Dance Studio   

   20th Most Common Venue  
29           Cupcake Shop

In [79]:
cluster= 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
4  East Toronto               4                 Trail     Health Food Store   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
4                   Pub           Yoga Studio                 Diner   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
4        Discount Store   Distribution Center               Dog Run   

  9th Most Common Venue 10th Most Common Venue 11th Most Common Venue  \
4      Doner Restaurant             Donut Shop           Dessert Shop   

  12th Most Common Venue       13th Most Common Venue 14th Most Common Venue  \
4    Dumpling Restaurant  Eastern European Restaurant      Electronics Store   

  15th Most Common Venue 16th Most Common Venue 17th Most Common Venue  \
4   Ethiopian Restaurant            Event Space     Dim Sum Restaurant   

  18th Most Common Venue 19th Most Common Venue 20th Most Common Venue  
4          Deli / Bodega       Department Store         Farmers Market

In [80]:
cluster= 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
19  Central Toronto               5        Ice Cream Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
19          Home Service                Garden                 Diner   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
19        Discount Store   Distribution Center               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
19      Doner Restaurant           Yoga Studio     Dim Sum Restaurant   

   11th Most Common Venue       12th Most Common Venue 13th Most Common Venue  \
19    Dumpling Restaurant  Eastern European Restaurant      Electronics Store   

   14th Most Common Venue 15th Most Common Venue 16th Most Common Venue  \
19   Ethiopian Restaurant            Event Space     Falafel Restaurant   

   17th Most Common Venue 18th Most Common Venue 19th Most Common Venue  \
19             Donut Shop           Dessert Shop   Fast Food Restaurant   

   20th Most Common Venue  
19       Department Store

In [81]:
cluster= 6
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
18  Central Toronto               6                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
18           Swim School              Bus Line        Farmers Market   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
18           Event Space  Ethiopian Restaurant     Electronics Store   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
18  Eastern European Restaurant   Dumpling Restaurant             Donut Shop   

   11th Most Common Venue 12th Most Common Venue 13th Most Common Venue  \
18       Doner Restaurant                Dog Run    Distribution Center   

   14th Most Common Venue 15th Most Common Venue 16th Most Common Venue  \
18         Discount Store                  Diner     Dim Sum Restaurant   

   17th Most Common Venue 18th Most Common Venue 19th Most Common Venue  \
18           Dessert Shop       Department Store          Deli / Bodega   

   20th Most Common Venue  
18           Dance Studio

In [82]:
cluster= 7
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
21  Central Toronto               7         Jewelry Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
21                 Trail      Sushi Restaurant    Mexican Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
21           Yoga Studio                 Diner           Event Space   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  \
21  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

   11th Most Common Venue 12th Most Common Venue 13th Most Common Venue  \
21    Dumpling Restaurant             Donut Shop       Doner Restaurant   

   14th Most Common Venue 15th Most Common Venue 16th Most Common Venue  \
21                Dog Run    Distribution Center         Discount Store   

   17th Most Common Venue 18th Most Common Venue 19th Most Common Venue  \
21     Dim Sum Restaurant         Farmers Market           Dessert Shop   

   20th Most Common Venue  
21       Department Store

In [83]:
cluster= 8
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
32  Downtown Toronto               8       Airport Service   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
32        Airport Lounge      Airport Terminal   Rental Car Location   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
32       Harbor / Marina                 Plane         Boat or Ferry   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
32              Boutique                   Bar       Sculpture Garden   

   11th Most Common Venue 12th Most Common Venue 13th Most Common Venue  \
32           Airport Gate     Airport Food Court                Airport   

   14th Most Common Venue       15th Most Common Venue 16th Most Common Venue  \
32            Coffee Shop  Eastern European Restaurant       Doner Restaurant   

   17th Most Common Venue 18th Most Common Venue 19th Most Common Venue  \
32             Donut Shop    Dumpling Restaurant            Yoga Studio   

   20th Most Common Venue  
32      Electronics Store

In [85]:
cluster= 9
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
20  Central Toronto               9                   Gym   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
20                 Hotel                  Park     Food & Drink Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
20        Sandwich Place      Department Store        Breakfast Spot   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  \
20  Gym / Fitness Center           Pizza Place      Convention Center   

   11th Most Common Venue 12th Most Common Venue       13th Most Common Venue  \
20             Comic Shop      Electronics Store  Eastern European Restaurant   

   14th Most Common Venue 15th Most Common Venue 16th Most Common Venue  \
20    Dumpling Restaurant             Donut Shop       Doner Restaurant   

   17th Most Common Venue 18th Most Common Venue 19th Most Common Venue  \
20                Dog Run    Distribution Center         Discount Store   

   20th Most Common Venue  
20      Convenience Store